In [ ]:
import torch.nn as nn
import torch
from torch import Tensor
from torch.nn import Transformer
import math
import numpy as np
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

### Решение вдохновлено https://pytorch.org/tutorials/beginner/translation_transformer.html

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from pathlib import Path
Path('/kaggle/working/model').mkdir(parents=True, exist_ok=True)

In [ ]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [ ]:
SRC_LANGUAGE = 'de'
TRGT_LANGUAGE = 'en'
token_transform = {}
token_transform[SRC_LANGUAGE] = get_tokenizer(None, language='de_core_web_sm')
token_transform[TRGT_LANGUAGE] = get_tokenizer(None, language='en_core_web_sm')

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, source, target, data_type = 'train'):
        self.type = data_type
        self.source_lines = None
        with open(source, 'r') as file:
            self.source_lines = file.readlines()
        if self.type != 'test': 
            self.taret_lines = None
            with open(target, 'r') as file:
                self.taret_lines = file.readlines()

        
    def __len__(self):
        return len(self.source_lines)
    
    def __getitem__(self, idx):
        if self.type != 'test':
            return (self.source_lines[idx], self.taret_lines[idx])
        else:
            return self.source_lines[idx]


In [ ]:
train_dataset = TranslationDataset("/kaggle/input/bhw2-data2/train.de-en.de", "/kaggle/input/bhw2-data2/train.de-en.en")
valid_dataset = TranslationDataset("/kaggle/input/bhw2-data2/val.de-en.de", "/kaggle/input/bhw2-data2/val.de-en.en")

In [ ]:
def yield_tokens(data_iter, language):
    language_index = {SRC_LANGUAGE: 0, TRGT_LANGUAGE: 1}
    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

vocab_transform = {}
for ln in [SRC_LANGUAGE, TRGT_LANGUAGE]:
    tokens = yield_tokens(train_dataset, ln)
    vocab_transform[ln] = build_vocab_from_iterator(
        tokens,
        min_freq=1,
        specials=special_symbols,
        special_first=True,
    )

for ln in [SRC_LANGUAGE, TRGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)


In [ ]:
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def tensor_transform(token_ids):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

text_transform = {}
for ln in [SRC_LANGUAGE, TRGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], 
                                               vocab_transform[ln], 
                                               tensor_transform) 

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TRGT_LANGUAGE](tgt_sample.rstrip("\n")))
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=True)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX, batch_first=True)
    return src_batch, tgt_batch


In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[1]
    tgt_seq_len = tgt.shape[1]
    
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)
    
    src_padding_mask = (src == PAD_IDX)
    tgt_padding_mask = (tgt == PAD_IDX)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [ ]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)


In [ ]:
class Seq2SeqTransformer(nn.Module):
    def __init__(
        self,
        num_encoder_layers: int,
        num_decoder_layers: int,
        emb_size: int,
        nhead: int,
        src_vocab_size: int,
        tgt_vocab_size: int,
        dim_feedforward: int = 512,
        dropout: float = 0.1
    ):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(
            d_model=emb_size,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)
    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)
    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)
    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [ ]:
def save_model(model, optimizer, scheduler = None, PATH = "/kaggle/working/model/checkpoint.pth"):
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict' : scheduler.state_dict() if scheduler else {},
            }, PATH)

In [ ]:
@torch.no_grad()
def evaluate(model, criterion, val_loader, tqdm_desc):
    model.eval()
    losses = 0
    for src, tgt in tqdm(val_loader, total=len(list(val_loader)), desc= tqdm_desc):
        src = src.to(device)
        tgt = tgt.to(device)
    
        tgt_input = tgt[:, :-1]
        
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        
        logits = model(
            src, 
            tgt_input, 
            src_mask, 
            tgt_mask,
            src_padding_mask, 
            tgt_padding_mask, 
            src_padding_mask
        )
        tgt_out = tgt[:, 1:]
        loss = criterion(logits.view(-1, tgt_vocab_size), tgt_out.contiguous().view(-1))
        losses += loss.item()
        torch.cuda.empty_cache()
    return losses / len(list(val_loader))


def train_epoch(model, optimizer, criterion, train_loader, tqdm_desc):
    model.train()
    losses = 0
    for src, tgt in tqdm(train_loader, total=len(list(train_loader)), desc = tqdm_desc):            
        src = src.to(device)
        tgt = tgt.to(device)
        
        tgt_input = tgt[:, :-1]
        
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = model(
            src, 
            tgt_input, 
            src_mask, 
            tgt_mask,
            src_padding_mask, 
            tgt_padding_mask, 
            src_padding_mask
        )
        optimizer.zero_grad()
        tgt_out = tgt[:, 1:]
        loss = criterion(logits.view(-1, tgt_vocab_size), tgt_out.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        losses += loss.item()
        torch.cuda.empty_cache()
    return losses / len(list(train_loader))
    


def train(model, optimizer, criterion, scheduler, train_loader, val_loader, n_epochs):
    train_loss = 0.0
    val_loss = 0.0
    best_val_loss = 1e8
    for epoch in range(n_epochs):
        train_loss = train_epoch(model, optimizer, criterion, train_loader,  f'Training epoch {epoch + 1}/{n_epochs}')
        val_loss = evaluate(model, criterion, val_loader, f'Validating epoch {epoch + 1}/{n_epochs}')
        print(f"Epoch {epoch + 1}")
        print(f" train loss: {train_loss}")
        print(f" val loss: {val_loss}\n")
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            save_model(model, optimizer, scheduler, PATH = "/kaggle/working/model/best_model.pth")
            print('Saved better model')
        wandb.log({'train_loss': train_loss, 'val_loss': val_loss})
        if scheduler is not None:
            scheduler.step(val_loss)

In [ ]:
def load_model(model, optimizer, scheduler = None, PATH = "/kaggle/working/model/checkpoint.pth"):
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    return model, optimizer, scheduler

In [ ]:
src_vocab_size = len(vocab_transform[SRC_LANGUAGE])
tgt_vocab_size = len(vocab_transform[TRGT_LANGUAGE])
emb_size = 1024
num_head = 8
hidden_dim = 1024
batch_size = 128
num_enc_layers = 3
num_dec_layers = 3


In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn)
val_dataloader = DataLoader(valid_dataset, batch_size=batch_size, collate_fn=collate_fn)

In [ ]:
import wandb

run = wandb.init(project="bhw2", entity = "polina-kadeyshvili")

In [ ]:
model = Seq2SeqTransformer(
    num_enc_layers, 
    num_dec_layers, 
    emb_size,
    num_head, 
    src_vocab_size, 
    tgt_vocab_size, 
    hidden_dim
).to(device)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

In [ ]:
n_epochs = 15


criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX, label_smoothing = 0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)
scheduler = None

In [ ]:
train(model, optimizer, criterion, scheduler, train_dataloader, val_dataloader, n_epochs)

In [ ]:
wandb.finish()

In [ ]:
loaded_model = torch.load('/kaggle/working/model/best_model.pth')
best_model = Seq2SeqTransformer(
    num_enc_layers, 
    num_dec_layers, 
    emb_size,
    num_head, 
    src_vocab_size, 
    tgt_vocab_size, 
    hidden_dim
).to(device)
best_model.load_state_dict(loaded_model['model_state_dict'])


In [ ]:
def beam_search_decode(model, src, src_mask, max_len, start_symbol, beam_size):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    
    beam = [(ys, 0)]
    
    for i in range(max_len-1):
        candidates = []
        
        for ys, score in beam:
            memory = memory.to(device)
            if i == 0:
                ys = ys.transpose(1, 0)
            tgt_mask = (generate_square_subsequent_mask(ys.size(1))
                        .type(torch.bool)).to(device)
            out = model.decode(ys, memory, tgt_mask)
            out = out
            prob = model.generator(out[:, -1])
            _, topi = torch.topk(prob, beam_size)
            
            for k in range(beam_size):
                next_word = topi[0][k].item()
                new_ys = torch.cat([ys,
                                    torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
                candidates.append((new_ys, score + torch.log(prob[0][next_word]).item()))
        
        candidates.sort(key=lambda x: x[1], reverse=True)
        
        beam = candidates[:beam_size]
        
        done = True
        for ys, score in beam:
            if ys[0][-1].item() != EOS_IDX:
                done = False
                break
        if done:
            break
    
    beam.sort(key=lambda x: x[1], reverse=True)
    
    return beam[0][0]

def translate_beam_search(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(1, -1)
    num_tokens = src.shape[1]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = beam_search_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX, beam_size=5).flatten()
    return " ".join(vocab_transform[TRGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        if i == 0:
            ys = ys.transpose(1, 0)
        tgt_mask = (generate_square_subsequent_mask(ys.size(1))
                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()    
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
        if next_word == EOS_IDX:
            break
    return ys


def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(1, -1)
    num_tokens = src.shape[1]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TRGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [ ]:
test_data = TranslationDataset('/kaggle/input/bhw2-data2/test1.de-en.de', None, data_type = 'test')

In [ ]:
with open('results.txt', 'w') as f:
    for sentence in test_data:
        res = translate(best_model, sentence)
        res += '\n'
        f.write(res)